In [105]:
from pathlib import Path 
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce
import shutil
import datetime
from datetime import date


sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm0104",
    security_token="EwVPSL02qXM8XQ2pkUdA13aW",
)

Descarga de pacientes y empresas

In [55]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  select Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c  from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
    ]
]
Empresas_SF.shape

(7922, 7)

In [56]:
# recuperar todos los pacientes ID desdde aAccount
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc,Phone  from Account where ispersonaccount = True
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
        'Phone'
    ]
]
Pacientes_SF.shape

(1534753, 9)

In [57]:
# recuperar todos los pacientes ID desde Contact
#results = sf.query_all(
#    """
#   select Id,Name,AccountId  from Contact
#    """
#)
#Pacientesc_SF = pd.DataFrame(results["records"])

#Pacientesc_SF = Pacientesc_SF[["Id", "AccountId", "Name"]]
#Pacientesc_SF.shape

In [58]:
#Relacion AccountContactRelacion se descarga la relacion paciente Empresa en account contact no es la q se ve
#results = sf.query_all(
#    """
#   select Id,AccountId,ContactId,NameAccount__c, RutContact__c from AccountContactRelation
#    """
#)
#Relacion_Pacientes_Empresas = pd.DataFrame(results["records"])

#Relacion_Pacientes_Empresas = Relacion_Pacientes_Empresas[
#    ["Id", "AccountId", "ContactId", "NameAccount__c", "RutContact__c"]
#]
#Relacion_Pacientes_Empresas.shape

In [59]:
#union de pacientes
Union_account_contact_relation = pd.merge(
    left=Relacion_Pacientes_Empresas,
    right=Pacientesc_SF,
    how="left",
    left_on="ContactId",
    right_on="Id",
)
Union_account_contact_relation.head()

,Id_x,AccountId_x,ContactId,NameAccount__c,RutContact__c,Id_y,AccountId_y,Name
0,07k8c00000RGWPAAA5,0018c00002axQdlAAE,0038c000036jelbAAA,SKBerge,182204846,0038c000036jelbAAA,0018c00002cVWElAAO,Maldonado Velasquez Nicolas
1,07k8c00000RGWPBAA5,0018c00002axQdlAAE,0038c000035CWLYAA4,SKBerge,18396206K,0038c000035CWLYAA4,0018c00002amMfoAAE,Zaror Yunis Daniel Esteban
2,07k8c00000RGWPCAA5,0018c00002axQdlAAE,0038c000035CaxiAAC,SKBerge,184262452,0038c000035CaxiAAC,0018c00002amRHeAAM,Bustamante Morales Flor Del Carmen
3,07k8c00000RGWPDAA5,0018c00002axQdlAAE,0038c000036jelcAAA,SKBerge,184429772,0038c000036jelcAAA,0018c00002cVWEmAAO,Malleo Abarca Rigoberto Andres
4,07k8c00000RGWPKAA5,0018c00002axQa5AAE,0038c000036jLDoAAM,Finning,233986291,0038c000036jLDoAAM,0018c00002cVCmMAAW,Albanez Gonzalez Matias Oliver Franci


In [60]:
#union de empresas
Union_account_contact_relation = pd.merge(
    left=Union_account_contact_relation,
    right=Empresas_SF,
    how="left",
    left_on="AccountId_x",
    right_on="Id",
)
Union_account_contact_relation.head()

,Id_x,AccountId_x,ContactId,NameAccount__c,RutContact__c,Id_y,AccountId_y,Name_x,Id,Name_y,Holding__c,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c
0,07k8c00000RGWPAAA5,0018c00002axQdlAAE,0038c000036jelbAAA,SKBerge,182204846,0038c000036jelbAAA,0018c00002cVWElAAO,Maldonado Velasquez Nicolas,0018c00002axQdlAAE,SKBerge,Sin Holding Asociado,MetLife,EMP5641,SKBerge,340013107
1,07k8c00000RGWPBAA5,0018c00002axQdlAAE,0038c000035CWLYAA4,SKBerge,18396206K,0038c000035CWLYAA4,0018c00002amMfoAAE,Zaror Yunis Daniel Esteban,0018c00002axQdlAAE,SKBerge,Sin Holding Asociado,MetLife,EMP5641,SKBerge,340013107
2,07k8c00000RGWPCAA5,0018c00002axQdlAAE,0038c000035CaxiAAC,SKBerge,184262452,0038c000035CaxiAAC,0018c00002amRHeAAM,Bustamante Morales Flor Del Carmen,0018c00002axQdlAAE,SKBerge,Sin Holding Asociado,MetLife,EMP5641,SKBerge,340013107
3,07k8c00000RGWPDAA5,0018c00002axQdlAAE,0038c000036jelcAAA,SKBerge,184429772,0038c000036jelcAAA,0018c00002cVWEmAAO,Malleo Abarca Rigoberto Andres,0018c00002axQdlAAE,SKBerge,Sin Holding Asociado,MetLife,EMP5641,SKBerge,340013107
4,07k8c00000RGWPKAA5,0018c00002axQa5AAE,0038c000036jLDoAAM,Finning,233986291,0038c000036jLDoAAM,0018c00002cVCmMAAW,Albanez Gonzalez Matias Oliver Franci,0018c00002axQa5AAE,Finning,Sin Holding Asociado,MetLife,EMP5951,Finning,340007528


In [61]:
#union de pacientes con sus datos
Union_account_contact_relation = pd.merge(
    left=Union_account_contact_relation,
    right=Pacientes_SF,
    how="left",
    left_on="RutContact__c",
    right_on="IdentificationId__pc",
)
Union_account_contact_relation.head()

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_26320\3766479298.py:2: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Id_x'} in the result is deprecated and will raise a MergeError in a future version.
  Union_account_contact_relation = pd.merge(


,Id_x,AccountId_x,ContactId,NameAccount__c,RutContact__c,Id_y,AccountId_y,Name_x,Id_x,Name_y,...,poliza__c,Id_y,IdentificationId__pc,Name,Identification_Type__c,Ocupaci_n__c,PersonTitle,PersonDepartment,HealthCloudGA__Gender__pc,Phone
0,07k8c00000RGWPAAA5,0018c00002axQdlAAE,0038c000036jelbAAA,SKBerge,182204846,0038c000036jelbAAA,0018c00002cVWElAAO,Maldonado Velasquez Nicolas,0018c00002axQdlAAE,SKBerge,...,340013107,0018c00002cVWElAAO,182204846,Maldonado Velasquez Nicolas,RUT,None,None,None,Hombre,None
1,07k8c00000RGWPBAA5,0018c00002axQdlAAE,0038c000035CWLYAA4,SKBerge,18396206K,0038c000035CWLYAA4,0018c00002amMfoAAE,Zaror Yunis Daniel Esteban,0018c00002axQdlAAE,SKBerge,...,340013107,0018c00002amMfoAAE,18396206K,Zaror Yunis Daniel Esteban,RUT,None,None,None,Hombre,None
2,07k8c00000RGWPCAA5,0018c00002axQdlAAE,0038c000035CaxiAAC,SKBerge,184262452,0038c000035CaxiAAC,0018c00002amRHeAAM,Bustamante Morales Flor Del Carmen,0018c00002axQdlAAE,SKBerge,...,340013107,0018c00002amRHeAAM,184262452,Bustamante Morales Flor Del Carmen,RUT,None,None,None,Mujer,None
3,07k8c00000RGWPDAA5,0018c00002axQdlAAE,0038c000036jelcAAA,SKBerge,184429772,0038c000036jelcAAA,0018c00002cVWEmAAO,Malleo Abarca Rigoberto Andres,0018c00002axQdlAAE,SKBerge,...,340013107,0018c00002cVWEmAAO,184429772,Malleo Abarca Rigoberto Andres,RUT,None,None,None,Hombre,None
4,07k8c00000RGWPKAA5,0018c00002axQa5AAE,0038c000036jLDoAAM,Finning,233986291,0038c000036jLDoAAM,0018c00002cVCmMAAW,Albanez Gonzalez Matias Oliver Franci,0018c00002axQa5AAE,Finning,...,340007528,0018c00002cVCmMAAW,233986291,Albanez Gonzalez Matias Oliver Franci,RUT,None,None,None,Hombre,None


********************************************************************************************************************************************************************

Aca las empresas que se ven de pacientes programas y programas por empresa


In [62]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

(7926, 5)

Aca se cruza programas por empresa mas empresas sf

In [63]:


Empresas_prog_y_empre = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Empresas_prog_y_empre.shape

(7930, 12)

In [64]:
Empresas_prog_y_empre.to_excel(
    "C:/Users/jmartinez/Documents/Empresas_final.xlsx", index=False
)

In [65]:
#buscar empresas desde Paciente_Programas__c relaciones de empresas con pacientes
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF.shape

(1230730, 4)

In [66]:
#unir empresas y todas sus caracteristicas programas pacientes cruce de pacientes con sus empresas



empresas_unidas = pd.merge(
    left=Paciente_Programas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Id",
)
empresas_unidas.shape

(1230730, 9)

In [67]:
empresas_unidas.head()

,Id_x,Paciente__c,Programas_por_Empresas__c,padre__c,Id_y,Empresa__c,Empresa_Padre__c,Nombre_Empresa__c,Name
0,a1Y8c00000C5asnEAB,0018c00002amLbpAAE,a1W8c000007vDPvEAM,MetLife,a1W8c000007vDPvEAM,0018c00002axQaeAAE,MetLife,Metlife Holding Metlife Serv. De Comerc Y Ases...,A-0173
1,a1Y8c00000C5asoEAB,0018c00002amLbqAAE,a1W8c000007vDSpEAM,MetLife,a1W8c000007vDSpEAM,0018c00002axQecAAE,MetLife,Clínica Alemana,A-0353
2,a1Y8c00000C5aspEAB,0018c00002amLbrAAE,a1W8c000007vDSpEAM,MetLife,a1W8c000007vDSpEAM,0018c00002axQecAAE,MetLife,Clínica Alemana,A-0353
3,a1Y8c00000C5asqEAB,0018c00002amLbsAAE,a1W8c000007vDSpEAM,MetLife,a1W8c000007vDSpEAM,0018c00002axQecAAE,MetLife,Clínica Alemana,A-0353
4,a1Y8c00000C5asrEAB,0018c00002amLc2AAE,a1W8c000007vDSpEAM,MetLife,a1W8c000007vDSpEAM,0018c00002axQecAAE,MetLife,Clínica Alemana,A-0353


In [68]:
uni_pacientes = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
    ]
]


uni_empresas = empresas_unidas[
    [
        "Id_x",
        "Paciente__c",
        "Programas_por_Empresas__c",
        "padre__c",
        "Id_y",
        "Empresa__c",
        "Empresa_Padre__c",
        "Nombre_Empresa__c",
        "Name",
    ]
]
pacientessf_empresas = pd.merge(
    left=uni_empresas,
    right=uni_pacientes,
    how="left",
    left_on="Paciente__c",
    right_on="Id",
)
pacientessf_empresas.shape

(1230730, 17)

In [69]:
pacientessf_empresas.head()


,Id_x,Paciente__c,Programas_por_Empresas__c,padre__c,Id_y,Empresa__c,Empresa_Padre__c,Nombre_Empresa__c,Name_x,Id,IdentificationId__pc,Name_y,Identification_Type__c,Ocupaci_n__c,PersonTitle,PersonDepartment,HealthCloudGA__Gender__pc
0,a1Y8c00000C5asnEAB,0018c00002amLbpAAE,a1W8c000007vDPvEAM,MetLife,a1W8c000007vDPvEAM,0018c00002axQaeAAE,MetLife,Metlife Holding Metlife Serv. De Comerc Y Ases...,A-0173,0018c00002amLbpAAE,224488556,Genesis Antonia Gutierrez Abarca,RUT,None,None,None,Mujer
1,a1Y8c00000C5asoEAB,0018c00002amLbqAAE,a1W8c000007vDSpEAM,MetLife,a1W8c000007vDSpEAM,0018c00002axQecAAE,MetLife,Clínica Alemana,A-0353,0018c00002amLbqAAE,113376066,Claudia Silva,RUT,None,None,None,Mujer
2,a1Y8c00000C5aspEAB,0018c00002amLbrAAE,a1W8c000007vDSpEAM,MetLife,a1W8c000007vDSpEAM,0018c00002axQecAAE,MetLife,Clínica Alemana,A-0353,0018c00002amLbrAAE,79476161,Maria Lorenza Garay,RUT,None,None,None,Mujer
3,a1Y8c00000C5asqEAB,0018c00002amLbsAAE,a1W8c000007vDSpEAM,MetLife,a1W8c000007vDSpEAM,0018c00002axQecAAE,MetLife,Clínica Alemana,A-0353,0018c00002amLbsAAE,181854227,Estolaza Vidal Jean Patrick Moises,RUT,None,None,None,Hombre
4,a1Y8c00000C5asrEAB,0018c00002amLc2AAE,a1W8c000007vDSpEAM,MetLife,a1W8c000007vDSpEAM,0018c00002axQecAAE,MetLife,Clínica Alemana,A-0353,0018c00002amLc2AAE,126899823,Rivas Correa Claudia,RUT,None,None,None,Mujer


ACA tenemos toda la relacion de pacientes y empresas del sistema

In [70]:
#empresas_unidas.head()
#apprix_1 = empresas_unidas.iloc[:611178,:]
#apprix_2 = empresas_unidas.iloc[611178:,:]
#apprix_1.to_excel(
#    "C:/Users/jmartinez/Documents/empresasA.xlsx", index=False
#)
#apprix_2.to_excel(
#    "C:/Users/jmartinez/Documents/empresasB.xlsx", index=False
#)

Descarga desde Citas de Servicios

In [71]:
# Citas de Servicio
results = sf.query_all(
    """
    SELECT Id, clinica__c,AccountId,CreatedDate,RUT__c,Work_TypeGroup__c,Status,Interacci_n__c,AppointmentNumber,Tipo_de_la_cita__c from ServiceAppointment
    """
)


Citas_de_servicio = pd.DataFrame(results["records"])
Citas_de_servicio['Empresas'] = np.nan
#Citas_de_servicio['RUT__c'].map(pacientessf_empresas.set_index('IdentificationId__pc_x',verify_integrity=False)['Name']) 
Citas_de_servicio['Empresas Id'] = np.nan
Citas_de_servicio = Citas_de_servicio[
    [
        "AppointmentNumber",        
        "RUT__c",  
        'Empresas',      
        "CreatedDate",
        "Tipo_de_la_cita__c",
        'Empresas Id',        
        "AccountId",
        'Clinica__c'
        
    ]
]
Citas_de_servicio.shape


(7466, 8)

In [72]:
# Citas de Servicio Orientacion
results = sf.query_all(
    """
    SELECT Id, AccountId,CreatedDate,RUT__c,WorkTypeId,Work_TypeGroup__c,Status,Interacci_n__c,Tipo_de_la_cita__c,AppointmentNumber from ServiceAppointment
    """
)
Citas_de_servicio2 = pd.DataFrame(results["records"])
Citas_de_servicio2['Clinica__c'] = 'Orientación Médica'
Citas_de_servicio2['Empresas'] = np.nan
Citas_de_servicio2['Empresas Id'] = np.nan
Citas_de_servicio2 = Citas_de_servicio2[
    [
        "AppointmentNumber",        
        "RUT__c",  
        'Empresas',      
        "CreatedDate",
        "Tipo_de_la_cita__c",
        'Empresas Id',       
        "AccountId",
        'Clinica__c',
        
    ]
]
Citas_de_servicio2.shape


(7466, 8)

In [73]:
#unir dos citas
Gran_Citas_deservicio = pd.concat([Citas_de_servicio, Citas_de_servicio2], axis=0)
Gran_Citas_deservicio = Gran_Citas_deservicio.rename(
    columns={
        "AppointmentNumber": "Codigo Ficha",
        "RUT__c": "Rut",
        "Empresas": "Empleador",
        "CreatedDate": "Fecha",
        "Tipo_de_la_cita__c": "Canal",
        "Empresas Id": "Cod. Empresa",
        "AccountId": "Id Paciente",        
        'Clinica__c':'Clínica'
    }
)

Gran_Citas_deservicio.shape

(14932, 8)

In [74]:
Gran_Citas_deservicio.head()

,Codigo Ficha,Rut,Empleador,Fecha,Canal,Cod. Empresa,Id Paciente,Clínica
0,SA-3124,183552252,NaN,2022-12-02T16:23:05.000+0000,Phone,NaN,0018c00002azpJ4AAI,Clínica de Nutrición
1,SA-8295,12654586K,NaN,2023-01-30T12:32:08.000+0000,Phone,NaN,0018c00002amTlVAAU,Clínica de Deporte
2,SA-4987,224488556,NaN,2022-12-21T21:28:49.000+0000,Phone,NaN,0018c00002amLbpAAE,Clínica de Nutrición
3,SA-3905,185384799,NaN,2022-12-12T15:34:28.000+0000,Phone,NaN,0018c00002ay6KFAAY,Clínica de Deporte
4,SA-3871,187821444,NaN,2022-12-12T12:21:02.000+0000,Phone,NaN,0018c00002bI5vJAAS,Clínica de Psicología


Programas descargas

In [75]:
#Clinica_deporte
results = sf.query_all(
    """
   SELECT id,Name,Rut__c,Nombre_Empresa__c,CreatedDate,Canal__c, Empresa__c, Account__c from Ficha_Cl_nica_Deporte__c
     """
)
ficha_deporte = pd.DataFrame(results["records"])

ficha_deporte = ficha_deporte[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        
    ]
]
ficha_deporte['clinica'] = 'Deporte'
ficha_deporte = ficha_deporte.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",        
        'clinica':'Clínica'
    }
)
ficha_deporte.shape


(788, 8)

In [76]:
# Clinica Nutricion
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c from Ficha_Clinica_Nutrici_n__c 
     """
)
ficha_nutricion = pd.DataFrame(results["records"])

ficha_nutricion = ficha_nutricion[
    [
        "Name",
        "RUT__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        
    ]
]
ficha_nutricion["clinica"] = "Nutrición"

ficha_nutricion = ficha_nutricion.rename(
    columns={
        "Name": "Codigo Ficha",
        "RUT__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'Clínica'
    }
)


ficha_nutricion.shape

(1824, 8)

In [77]:
#Clinica AppSex
results = sf.query_all(
    """
   select  Name, Rut__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c, RecordTypeId from Ficha_Cl_nica_ApsSex__c 
     """
)
ficha_Apssex = pd.DataFrame(results["records"])

ficha_Apssex = ficha_Apssex[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        
    ]
]
ficha_Apssex['clinica'] = 'Asesoría Psicologíca en Salud Sexual'
ficha_Apssex = ficha_Apssex.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'Clínica'
    }
)

ficha_Apssex.shape

(3, 8)

In [106]:
# Clinica Psicologia
results = sf.query_all(
    """
   select  Name, Rut__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c, RecordTypeId,Motivo_de_consulta_manifiesto__c from Ficha_Cl_nica_Asistencia_Emocional__c 
     """
)
ficha_Psicologia = pd.DataFrame(results["records"])

ficha_Psicologia = ficha_Psicologia[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        "Motivo_de_consulta_manifiesto__c",
    ]
]
ficha_Psicologia["clinica"] = "Psicología"
ficha_Psicologia = ficha_Psicologia.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        "clinica": "Clínica",
        "Motivo_de_consulta_manifiesto__c": "Motivo de consulta",
    }
)
ficha_Psicologia.shape

(1338, 9)

In [79]:
#Clinica Sueño
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c, RecordTypeId from Ficha_Cl_nica_del_Sue_o__c 
     """
)
ficha_Sueno = pd.DataFrame(results["records"])
ficha_Sueno = ficha_Sueno[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
    ]
]
ficha_Sueno['clinica'] = 'Clínica Del Sueño'
ficha_Sueno = ficha_Sueno.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'Clínica'
    }
)
ficha_Sueno.shape

(193, 8)

In [80]:
#Clinica PAciente Cronico
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c from Ficha_Cl_nica_Seg_Paciente_Cr_nico__c  
     """
)
ficha_Cronico = pd.DataFrame(results["records"])
ficha_Cronico = ficha_Cronico[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
    ]
]
ficha_Cronico['clinica'] = 'Pacientes Crónicos'
ficha_Cronico = ficha_Cronico.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'Clínica'
    }
)
ficha_Cronico.shape

(678, 8)

In [81]:
#Clinica Telemedicina
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c from Ficha_Cl_nica_Telemedicina__c   
     """
)
ficha_Telemedicina = pd.DataFrame(results["records"])
ficha_Telemedicina = ficha_Telemedicina[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c"
    ]
]
ficha_Telemedicina['clinica'] = 'Telemedicina'
ficha_Telemedicina = ficha_Telemedicina.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",        
        'clinica':'Clínica'
    }
)
ficha_Telemedicina.shape

(2298, 8)

In [82]:
#Clinica Veterinaria
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c from Ficha_Cl_nica_Veterinaria__c    
     """
)
ficha_Veterinaria = pd.DataFrame(results["records"])
ficha_Veterinaria = ficha_Veterinaria[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c"
    ]
]
ficha_Veterinaria['clinica'] = 'Veterinaria'
ficha_Veterinaria = ficha_Veterinaria.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'Clínica'
    }
)
ficha_Veterinaria.shape

(8, 8)

In [83]:
#Clinica Orientacion
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c from Ficha_Orientaci_n_Medica__c     
     """
)
ficha_Orientacion = pd.DataFrame(results["records"])
ficha_Orientacion = ficha_Orientacion[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        
    ]
]
ficha_Orientacion['clinica'] = 'Orientación Médica'
ficha_Orientacion = ficha_Orientacion.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'Clínica'
    }
)
ficha_Orientacion.shape

(18, 8)

In [84]:
#Clinica Front
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c from Ficha_Front__c     
     """
)
ficha_Front = pd.DataFrame(results["records"])

ficha_Front = ficha_Front[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c"
    ]
]
ficha_Front['clinica'] = 'Orientación Médica'
ficha_Front = ficha_Front.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'Clínica'
    }
)
ficha_Front.shape

(148, 8)

In [85]:
#Clinica_general
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c,Account__c,   Programa__c   from Ficha_Clinica_General__c 
     """
)
ficha_general = pd.DataFrame(results["records"])

ficha_general = ficha_general[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        'Programa__c'
        
    ]
]

ficha_general = ficha_general.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",        
        'Programa__c':'Clínica'
    }
)
ficha_general.shape


(904, 8)

In [108]:
#Clinica Ingreso deportivo
results = sf.query_all(
    """
   select  Name, RUT__c, Nombre_Empresa__c, CreatedDate, Canal__c, Empresa__c, Account__c from Ficha_Cl_nica_Ingreso_Deportivo__c    
     """
)
ficha_Ingreso_dep = pd.DataFrame(results["records"])

ficha_Ingreso_dep = ficha_Ingreso_dep[
    [
        "Name",
        "Rut__c",
        "Nombre_Empresa__c",
        "CreatedDate",
        "Canal__c",
        "Empresa__c",
        "Account__c",
        
    ]
]
ficha_Ingreso_dep['clinica'] = 'Ingreso Deporte Seguro'
ficha_Ingreso_dep = ficha_Ingreso_dep.rename(
    columns={
        "Name": "Codigo Ficha",
        "Rut__c": "Rut",
        "Nombre_Empresa__c": "Empleador",
        "CreatedDate": "Fecha",
        "Canal__c": "Canal",
        "Empresa__c": "Cod. Empresa",
        "Account__c": "Id Paciente",
        'clinica':'Clínica'
    }
)
ficha_Ingreso_dep.shape

(200, 8)

In [109]:
Clinicas = pd.concat([ficha_Front, ficha_Ingreso_dep,ficha_Orientacion,ficha_Veterinaria,ficha_Telemedicina,ficha_Cronico,ficha_Sueno,ficha_Psicologia,ficha_Apssex,ficha_nutricion,ficha_deporte,ficha_general], axis=0)
Clinicas.shape

(8400, 9)

In [110]:
Clinicas.head()

,Codigo Ficha,Rut,Empleador,Fecha,Canal,Cod. Empresa,Id Paciente,Clínica,Motivo de consulta
0,FFF-0035,181523778,Farmacias Ahumada S.A.,2023-01-24T18:31:33.000+0000,Entrante - WhatsApp,a1Y8c00000C6VINEA3,0018c00002amVPcAAM,Orientación Médica,NaN
1,FFF-0088,164841448,Evaluaserve Chile S.A.,2023-01-26T14:25:58.000+0000,Entrante - Chat Online,a1Y8c00000Clq5uEAB,0018c00002b03a0AAA,Orientación Médica,NaN
2,FFF-0096,188075606,None,2023-01-26T17:29:57.000+0000,Entrante - Llamada,None,0018c00002eimgBAAQ,Orientación Médica,NaN
3,FFF-0076,195034931,Metlife Holding Metlife Sacyr Operacion Y Serv...,2023-01-26T12:17:12.000+0000,Entrante - Chat Online,a1Y8c00000CltMaEAJ,0018c00002b0aVyAAI,Orientación Médica,NaN
4,FFF-0037,184478412,Asociación Chilena De Seguridad,2023-01-24T19:19:01.000+0000,Saliente - Llamada,a1Y8c00000Ckvd5EAB,0018c00002ay08PAAQ,Orientación Médica,NaN


In [111]:
Reortes_general = pd.concat([Clinicas,Gran_Citas_deservicio ], axis=0)
Reortes_general.shape

(23332, 9)

In [112]:
Reortes_general.head()

,Codigo Ficha,Rut,Empleador,Fecha,Canal,Cod. Empresa,Id Paciente,Clínica,Motivo de consulta
0,FFF-0035,181523778,Farmacias Ahumada S.A.,2023-01-24T18:31:33.000+0000,Entrante - WhatsApp,a1Y8c00000C6VINEA3,0018c00002amVPcAAM,Orientación Médica,NaN
1,FFF-0088,164841448,Evaluaserve Chile S.A.,2023-01-26T14:25:58.000+0000,Entrante - Chat Online,a1Y8c00000Clq5uEAB,0018c00002b03a0AAA,Orientación Médica,NaN
2,FFF-0096,188075606,None,2023-01-26T17:29:57.000+0000,Entrante - Llamada,None,0018c00002eimgBAAQ,Orientación Médica,NaN
3,FFF-0076,195034931,Metlife Holding Metlife Sacyr Operacion Y Serv...,2023-01-26T12:17:12.000+0000,Entrante - Chat Online,a1Y8c00000CltMaEAJ,0018c00002b0aVyAAI,Orientación Médica,NaN
4,FFF-0037,184478412,Asociación Chilena De Seguridad,2023-01-24T19:19:01.000+0000,Saliente - Llamada,a1Y8c00000Ckvd5EAB,0018c00002ay08PAAQ,Orientación Médica,NaN


In [113]:
Reortes_general["Mes"] = pd.to_datetime(
    Reortes_general["Fecha"]
).dt.month

Reortes_general["Año"] = pd.to_datetime(
    Reortes_general["Fecha"]
).dt.year
Reortes_general["Dia"] = pd.to_datetime(
    Reortes_general["Fecha"]
).dt.day

# x = date(Citas_de_servicio['Dia'], Citas_de_servicio['Mes'], Citas_de_servicio['Año'] )
Reortes_general["Fecha_chilena"] = pd.to_datetime(
    Reortes_general["Fecha"], dayfirst=True, yearfirst=False, exact=True
)
Reortes_general["Fecha_chilena2"] = Reortes_general["Fecha_chilena"].dt.strftime("%d-%m-%Y")

clinicas_ = {
    "Clínica de Nutrición": "Nutrición",
    "Clínica de Deporte": "Deporte",
    "Clínica de Psicología": "Psicología",
    "Clínica Ingreso Deportivo Seguro": "Deporte",
    "Clínica del Sueño": "Clínica Del Sueño",
    "Clínica Seguimiento Pacientes Crónicos": "Pacientes Crónicos",
    'Orientación Médica':'Orientación Médica' ,
    'Asesoría Deportiva':'Deporte',
    'Asistencia Emocional':"Psicología",
    'Asistencia Nutricional':'Nutrición',
    'Clínica APSSex':'Asesoría Psicologíca en Salud Sexual',
    'Clínica Del Sueño':'Clínica Del Sueño',
    'Clínica Veterinaria':'Veterinaria',
    

    
}
Reortes_general.replace({"Clinica__c": clinicas_}, inplace=True)


Reortes_general.head()



,Codigo Ficha,Rut,Empleador,Fecha,Canal,Cod. Empresa,Id Paciente,Clínica,Motivo de consulta,Mes,Año,Dia,Fecha_chilena,Fecha_chilena2
0,FFF-0035,181523778,Farmacias Ahumada S.A.,2023-01-24T18:31:33.000+0000,Entrante - WhatsApp,a1Y8c00000C6VINEA3,0018c00002amVPcAAM,Orientación Médica,NaN,1,2023,24,2023-01-24 18:31:33+00:00,24-01-2023
1,FFF-0088,164841448,Evaluaserve Chile S.A.,2023-01-26T14:25:58.000+0000,Entrante - Chat Online,a1Y8c00000Clq5uEAB,0018c00002b03a0AAA,Orientación Médica,NaN,1,2023,26,2023-01-26 14:25:58+00:00,26-01-2023
2,FFF-0096,188075606,None,2023-01-26T17:29:57.000+0000,Entrante - Llamada,None,0018c00002eimgBAAQ,Orientación Médica,NaN,1,2023,26,2023-01-26 17:29:57+00:00,26-01-2023
3,FFF-0076,195034931,Metlife Holding Metlife Sacyr Operacion Y Serv...,2023-01-26T12:17:12.000+0000,Entrante - Chat Online,a1Y8c00000CltMaEAJ,0018c00002b0aVyAAI,Orientación Médica,NaN,1,2023,26,2023-01-26 12:17:12+00:00,26-01-2023
4,FFF-0037,184478412,Asociación Chilena De Seguridad,2023-01-24T19:19:01.000+0000,Saliente - Llamada,a1Y8c00000Ckvd5EAB,0018c00002ay08PAAQ,Orientación Médica,NaN,1,2023,24,2023-01-24 19:19:01+00:00,24-01-2023


In [114]:


Reortes_general['Fecha_chilena'] = Reortes_general['Fecha_chilena'].dt.tz_localize(None)

Reortes_general.Rut.fillna('0', inplace=True)
Reortes_general.to_excel(
    "C:/Users/jmartinez/Documents/reportes.xlsx", index=False
)

In [93]:
pacientessf_empresas.head()

,Id_x,Paciente__c,Programas_por_Empresas__c,padre__c,Id_y,Empresa__c,Empresa_Padre__c,Nombre_Empresa__c,Name_x,Id,IdentificationId__pc,Name_y,Identification_Type__c,Ocupaci_n__c,PersonTitle,PersonDepartment,HealthCloudGA__Gender__pc
0,a1Y8c00000C5asnEAB,0018c00002amLbpAAE,a1W8c000007vDPvEAM,MetLife,a1W8c000007vDPvEAM,0018c00002axQaeAAE,MetLife,Metlife Holding Metlife Serv. De Comerc Y Ases...,A-0173,0018c00002amLbpAAE,224488556,Genesis Antonia Gutierrez Abarca,RUT,None,None,None,Mujer
1,a1Y8c00000C5asoEAB,0018c00002amLbqAAE,a1W8c000007vDSpEAM,MetLife,a1W8c000007vDSpEAM,0018c00002axQecAAE,MetLife,Clínica Alemana,A-0353,0018c00002amLbqAAE,113376066,Claudia Silva,RUT,None,None,None,Mujer
2,a1Y8c00000C5aspEAB,0018c00002amLbrAAE,a1W8c000007vDSpEAM,MetLife,a1W8c000007vDSpEAM,0018c00002axQecAAE,MetLife,Clínica Alemana,A-0353,0018c00002amLbrAAE,79476161,Maria Lorenza Garay,RUT,None,None,None,Mujer
3,a1Y8c00000C5asqEAB,0018c00002amLbsAAE,a1W8c000007vDSpEAM,MetLife,a1W8c000007vDSpEAM,0018c00002axQecAAE,MetLife,Clínica Alemana,A-0353,0018c00002amLbsAAE,181854227,Estolaza Vidal Jean Patrick Moises,RUT,None,None,None,Hombre
4,a1Y8c00000C5asrEAB,0018c00002amLc2AAE,a1W8c000007vDSpEAM,MetLife,a1W8c000007vDSpEAM,0018c00002axQecAAE,MetLife,Clínica Alemana,A-0353,0018c00002amLc2AAE,126899823,Rivas Correa Claudia,RUT,None,None,None,Mujer


In [94]:

unir_reporte_con_empresas = pd.merge(
    left=Reortes_general,
    right=pacientessf_empresas,
    how="left",
    left_on="Rut",
    right_on="IdentificationId__pc",
)
unir_reporte_con_empresas.Empleador.fillna(unir_reporte_con_empresas.Nombre_Empresa__c , inplace=True)



In [101]:
unir_reporte_con_empresas = pd.merge(
    left=unir_reporte_con_empresas,
    right=Empresas_SF,
    how="left",
    left_on="Empresa__c",
    right_on="Id",
)

unir_reporte_con_empresas = pd.merge(
    left=unir_reporte_con_empresas,
    right=Pacientes_SF,
    how="left",
    left_on="Rut",
    right_on="IdentificationId__pc",
)


C:\Users\jmartinez\AppData\Local\Temp\ipykernel_26320\1160429987.py:1: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Name_x'} in the result is deprecated and will raise a MergeError in a future version.
  unir_reporte_con_empresas = pd.merge(
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_26320\1160429987.py:9: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Id_x'} in the result is deprecated and will raise a MergeError in a future version.
  unir_reporte_con_empresas = pd.merge(


In [102]:
unir_reporte_con_empresas.head()

,Codigo Ficha,Rut,Empleador,Fecha,Canal,Cod. Empresa,Id Paciente,Clínica,Mes,Año,...,poliza__c_y,Id_y,IdentificationId__pc_y,Name,Identification_Type__c_y,Ocupaci_n__c_y,PersonTitle_y,PersonDepartment_y,HealthCloudGA__Gender__pc_y,Phone
0,FFF-0035,181523778,Farmacias Ahumada S.A.,2023-01-24T18:31:33.000+0000,Entrante - WhatsApp,a1Y8c00000C6VINEA3,0018c00002amVPcAAM,Orientación Médica,1,2023,...,Sin Póliza Asociada,0018c00002amVPcAAM,181523778,Marjorie Melanie Rojas Cea,RUT,None,None,None,Mujer,956805279
1,FFF-0088,164841448,Evaluaserve Chile S.A.,2023-01-26T14:25:58.000+0000,Entrante - Chat Online,a1Y8c00000Clq5uEAB,0018c00002b03a0AAA,Orientación Médica,1,2023,...,340010253,0018c00002b03a0AAA,164841448,Maria Sofia Eberl Muñoz,RUT,None,None,None,Mujer,None
2,FFF-0096,188075606,NaN,2023-01-26T17:29:57.000+0000,Entrante - Llamada,None,0018c00002eimgBAAQ,Orientación Médica,1,2023,...,NaN,0018c00002eimgBAAQ,188075606,SATCHARY CARMONA GIACAMAN,RUT,None,None,None,Hombre,None
3,FFF-0076,195034931,Metlife Holding Metlife Sacyr Operacion Y Serv...,2023-01-26T12:17:12.000+0000,Entrante - Chat Online,a1Y8c00000CltMaEAJ,0018c00002b0aVyAAI,Orientación Médica,1,2023,...,None,0018c00002b0aVyAAI,195034931,Cristian Alexis Droguett Araya,RUT,None,None,None,Hombre,968355428
4,FFF-0037,184478412,Asociación Chilena De Seguridad,2023-01-24T19:19:01.000+0000,Saliente - Llamada,a1Y8c00000Ckvd5EAB,0018c00002ay08PAAQ,Orientación Médica,1,2023,...,340007825,0018c00002ay08PAAQ,184478412,Ana Belen Orellana Catalan,RUT,None,None,None,Mujer,957162954


In [103]:
unir_reporte_con_empresas.to_excel(
    "C:/Users/jmartinez/Documents/reportes2.xlsx", index=False
)

In [115]:
Metlife = unir_reporte_con_empresas[(unir_reporte_con_empresas['Empresa_Padre__c']=='MetLife')]
Metlife.to_excel(
    "C:/Users/jmartinez/Documents/Metlife.xlsx", index=False
)
Zurich = unir_reporte_con_empresas[(unir_reporte_con_empresas['Empresa_Padre__c']=='Zurich')]
Metlife.to_excel(
    "C:/Users/jmartinez/Documents/Zurich.xlsx", index=False
)
Security = unir_reporte_con_empresas[(unir_reporte_con_empresas['Empresa_Padre__c']=='Vida Security')]
Metlife.to_excel(
    "C:/Users/jmartinez/Documents/Vida Security.xlsx", index=False
)

PermissionError: [Errno 13] Permission denied: 'C:/Users/jmartinez/Documents/Metlife.xlsx'

In [99]:
nuevos = ("C:/Users/jmartinez/Documents/bechtel.xlsx")
Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].str.replace("-", "")
# aca se trabaja con el nuevo rut desde rut1 ojooooooo
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.head()

bechtel = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_SF,
    #right=pacientessf_empresas,
    how="left",
    left_on="rut",
    right_on="IdentificationId__pc",
)
Pacientes_SF

,Id,IdentificationId__pc,Name,Identification_Type__c,Ocupaci_n__c,PersonTitle,PersonDepartment,HealthCloudGA__Gender__pc,Phone
0,0018c00002amLTMAA2,173805004,Ximena Alejandra Muñoz Tudela,RUT,None,None,None,Mujer,981445038
1,0018c00002amLUGAA2,153338949,Juana Andrea Gallardo Espinoza,RUT,None,None,None,Mujer,942614540
2,0018c00002amLUHAA2,177735213,Valentina Jahdai Gonzalez Diaz,RUT,None,None,None,Mujer,967018325
3,0018c00002amLUIAA2,169332924,Deayan Nicole Parra Peralta,RUT,None,None,None,Mujer,999991691
4,0018c00002amLUJAA2,172957692,Camila Fernanda Velásquez Rojas,RUT,None,None,None,Mujer,943403432
...,...,...,...,...,...,...,...,...,...
1534748,0018c00002f372LAAQ,151668216,Marlene Andrea Fuentealba Avaria,RUT,None,None,None,Mujer,984221658
1534749,0018c00002f372zAAA,ZAXG590211HOCRXR07,Gregorio X Zarate,CURP,None,None,None,Hombre,None
1534750,0018c00002f374IAAQ,VECJ980113HSLLBS00,Jesús David Velázquez Ceballos,CURP,None,None,None,None,None
1534751,0018c00002f3CBZAA2,hesm180201mqtrtta8,Matías Santiago Hernandez Soto,CURP,None,None,None,Hombre,None


In [100]:
bechtel.to_excel(
    "C:/Users/jmartinez/Documents/bechtel_salida2.xlsx", index=False
)